## 6.4 baseline model

In [1]:
import numpy as np
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns
sns.set()

submission = pd.read_csv('/kaggle/input/bike-sharing-demand/sampleSubmission.csv')
train = pd.read_csv('/kaggle/input/bike-sharing-demand/train.csv')
test= pd.read_csv('/kaggle/input/bike-sharing-demand/test.csv')

In [2]:
train.shape, test.shape, submission.shape

((10886, 12), (6493, 9), (6493, 2))

test 에도 weather 에 4가 있다. 아래 all_data로 묶어서 삭제하니 y 타깃이 모자라는 현상 발생.\
찾아낸 이유 나는 합친 all_data에서 전처리를 하고 코드는 train 에서 y를 가져옴.\

최종. train, test 에서 weather 의 이상치를 제거해 주어도 submission 제출 파일의 모양이 test 파일과 같기 때문에\
train 파일에서만 이상치 제거를 시행해야 문제가 생기지 않는다.


In [3]:
train = train[train['weather'] != 4]
train.shape

(10885, 12)

### 6.4.1 feature engineering 

In [4]:
# 전처리 전 데이터 합치기
all_data = pd.concat([train, test], ignore_index=True)
all_data.head()

,datetime,season,holiday,workingday,weather,temp,atemp,humidity,windspeed,casual,registered,count
0,2011-01-01 00:00:00,1,0,0,1,9.84,14.395,81,0.0,3.0,13.0,16.0
1,2011-01-01 01:00:00,1,0,0,1,9.02,13.635,80,0.0,8.0,32.0,40.0
2,2011-01-01 02:00:00,1,0,0,1,9.02,13.635,80,0.0,5.0,27.0,32.0
3,2011-01-01 03:00:00,1,0,0,1,9.84,14.395,75,0.0,3.0,10.0,13.0
4,2011-01-01 04:00:00,1,0,0,1,9.84,14.395,75,0.0,0.0,1.0,1.0


### 파생 변수 추가

In [5]:
from datetime import datetime

all_data['date'] = all_data['datetime'].apply(lambda x: x.split()[0])
all_data['year'] = all_data['date'].apply(lambda x: x.split('-')[0])
all_data['month'] = all_data['date'].apply(lambda x: x.split('-')[1])
all_data['hour'] = all_data['datetime'].apply(lambda x: x.split()[1].split(':')[0])
all_data['weekday'] = all_data['date'].apply(lambda x: datetime.strptime(x, '%Y-%m-%d').weekday())
all_data.head()

,datetime,season,holiday,workingday,weather,temp,atemp,humidity,windspeed,casual,registered,count,date,year,month,hour,weekday
0,2011-01-01 00:00:00,1,0,0,1,9.84,14.395,81,0.0,3.0,13.0,16.0,2011-01-01,2011,01,00,5
1,2011-01-01 01:00:00,1,0,0,1,9.02,13.635,80,0.0,8.0,32.0,40.0,2011-01-01,2011,01,01,5
2,2011-01-01 02:00:00,1,0,0,1,9.02,13.635,80,0.0,5.0,27.0,32.0,2011-01-01,2011,01,02,5
3,2011-01-01 03:00:00,1,0,0,1,9.84,14.395,75,0.0,3.0,10.0,13.0,2011-01-01,2011,01,03,5
4,2011-01-01 04:00:00,1,0,0,1,9.84,14.395,75,0.0,0.0,1.0,1.0,2011-01-01,2011,01,04,5


In [6]:
all_data.columns

Index(['datetime', 'season', 'holiday', 'workingday', 'weather', 'temp',
       'atemp', 'humidity', 'windspeed', 'casual', 'registered', 'count',
       'date', 'year', 'month', 'hour', 'weekday'],
      dtype='object')

In [7]:
# 필요없는 컬럼 제거
all_data = all_data.drop(['windspeed', 'casual', 'registered', 'datetime', 'date', 'month'], axis=1)
all_data.head()

,season,holiday,workingday,weather,temp,atemp,humidity,count,year,hour,weekday
0,1,0,0,1,9.84,14.395,81,16.0,2011,00,5
1,1,0,0,1,9.02,13.635,80,40.0,2011,01,5
2,1,0,0,1,9.02,13.635,80,32.0,2011,02,5
3,1,0,0,1,9.84,14.395,75,13.0,2011,03,5
4,1,0,0,1,9.84,14.395,75,1.0,2011,04,5


### train test 로 다시 나누기

In [8]:
X_train = all_data[~pd.isnull(all_data['count'])]
X_test = all_data[pd.isnull(all_data['count'])]

# train data 에서 target 값인 count 제거
X_train = X_train.drop(['count'], axis=1)
X_test = X_test.drop(['count'], axis=1)

# y 값에 타깃 지정
# y = all_data['count']
# y = y.dropna()

y = train['count']

In [9]:
X_train.shape, y.shape

((10885, 10), (10885,))

### 6.4.2 평가지표 계산 함수 작성

In [10]:
def rmsle(y_true, y_pred, convertExp=True):
    # 지수변환
    if convertExp:
        y_true = np.exp(y_true)
        y_pred = np.exp(y_pred)
        
    # 로그변환 후 결측값을 0으로 변환
    log_true = np.nan_to_num(np.log(y_true+1))
    log_pred = np.nan_to_num(np.log(y_pred+1))
    
    # RMSLE 계산
    output = np.sqrt(np.mean((log_true - log_pred)**2))
    return output

### 6.4.3 모델 훈련

In [11]:
from sklearn.linear_model import LinearRegression

linear_reg_model = LinearRegression()

In [12]:
# 타깃값(count) log 변환 '''정규분포에 가깝게 변환해야 결과가 잘나온다.'''
log_y = np.log(y)
linear_reg_model.fit(X_train, log_y)

LinearRegression()

In [13]:
X_train.shape, log_y.shape

((10885, 10), (10885,))

### 6.4.4 모델 성능 검증

In [14]:
preds = linear_reg_model.predict(X_train)
print(f'선형회귀의 RMSLE 값 : {rmsle(log_y, preds, True):.4f}')

선형회귀의 RMSLE 값 : 1.0205


sklearn 에 있는 msle 함수에 sqrt 씌움, 위의 만든 함수와의 차이를 보고 싶어서...  그러나 실패.
```python
from sklearn.metrics import mean_squared_log_error
MSLE = mean_squared_log_error(log_y, preds)
RMSLE = np.sqrt(MSLE)

print('sklearn metrics MSLE :', MSLE)
print('sklearn metrics RMSLE :', RMSLE)
```

### 예측 및 결과 제출

In [15]:
linearreg_preds = linear_reg_model.predict(X_test) # 테스트 데이터로 예측

submission['count'] = np.exp(linearreg_preds)    # 지수변환
submission.to_csv('submission.csv', index=False) # 파일로 저장

In [16]:
X_test.shape, linearreg_preds.shape, submission.shape

# test 파일도 weather = 4 라는 이상치를 제거해줄 경우 제출하는 파일과 길이가 달라버리게 된다.
# 그러므로 wather = 4 를 가진 train 만 정리해야한다.

((6493, 10), (6493,), (6493, 2))

## 6.5 ~ 6.7 모델 성능 개선